# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
A. Patil  ->  A. Patil  |  ['A. Patil']
K. Doi  ->  K. Doi  |  ['K. Doi']


Arxiv has 65 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.07925
extracting tarball to tmp_2601.07925... done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'ATLASJ1013.bbl' from 'tmp_2601.07925/ATLASJ1013.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 41 bibliographic references in tmp_2601.07925/ATLASJ1013.bbl.
Retrieving document from  https://arxiv.org/e-print/2601.08087
extracting tarball to tmp_2601.08087... done.


A. Patil  ->  A. Patil  |  ['A. Patil']


Issues with the citations
repeated bibliography entry: 2022casa
Retrieving document from  https://arxiv.org/e-print/2601.08591
extracting tarball to tmp_2601.08591... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.07925-b31b1b.svg)](https://arxiv.org/abs/2601.07925) | **An eclipsing 8.56 minute orbital period mass-transferring binary**  |
|| E. T. Chickles, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2026-01-14*|
|*Comments*| *14 pages, 10 figures*|
|**Abstract**|            We report the discovery of ATLAS J101342.5-451646.8 (hereafter ATLAS J1013-4516), an eclipsing, mass-transferring AM Canum Venaticorum binary with an 8.56-minute orbital period, identified via periodic variability in light curves from the Asteroid Terrestrial-impact Last Alert System survey of Gaia white dwarf candidates. Follow-up spectroscopy with the Large Lenslet Array Magellan Spectrograph reveals a helium-dominated accretion disk, while high-speed photometry with ULTRACAM shows pronounced primary and secondary eclipses. We construct a decade-long timing baseline using data from ATLAS, Gaia, ULTRACAM on the Gran Telescopio Canarias, and the proto-Lightspeed instrument on the Magellan Clay telescope. From this baseline, we measure an orbital period derivative of dP/dt = -1.60 +/- 0.07 x 10^-12 seconds per second. Interpreted in the context of stable mass transfer, the magnitude and sign of the period derivative indicate that the orbital evolution is governed by the interplay between gravitational-wave-driven angular momentum losses and mass transfer, directly probing the donor star's structural response to mass loss. Assuming angular momentum loss dominated by gravitational wave emission, we constrain the component masses and infer the characteristic gravitational wave strain of the system for future space-based observatories such as the Laser Interferometer Space Antenna. We predict a characteristic strain corresponding to a four-year signal-to-noise ratio greater than approximately 20, establishing ATLAS J1013-4516 as a strong prospective source for probing long-term orbital evolution in the mass-transferring regime.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.08087-b31b1b.svg)](https://arxiv.org/abs/2601.08087) | **The Secret Lives of Open Clusters: a Multiwavelength Examination of Three Open Clusters**  |
|| K. C. Dage, et al. -- incl., <mark>A. Patil</mark> |
|*Appeared on*| *2026-01-14*|
|*Comments*| *accepted to PASA*|
|**Abstract**|            Star clusters are well known for their dynamical interactions, an outcome of their high stellar densities; in this paper we use multiwavelength observations to search for the unique outcomes of these interactions in three nearby Galactic open clusters: IC 2602 (30 Myr), NGC 2632 (750 Myr) and M67 (4 Gyr). We compared X-ray observations from all-sky surveys like eROSITA, plus archival observations from Chandra X-ray Observatory, survey radio observations from ASKAP's Evolutionary Map of the Universe survey plus archival VLA observations, in conjunction with new cluster catalogs with Gaia. From X-ray, we found 77 X-ray sources likely associated with IC 2602, 31 X-ray sources in NGC 2632, and 31 near M67's central regions. We were further able to classify these X-ray sources based on their optical variability and any radio emission. Three IC 2602 X-ray sources had radio counterparts, which are likely all chromospherically active binary stars. We also identified luminous radio and X-ray variability from a spectroscopic triple system in M67, WOCS 3012/S1077, which is either consistent with a quiescent black hole binary, or due to an active binary stellar system. A recent population study of optical variables by Anderson & Hunt 2025 shows that the population of optical variables in open clusters clearly changes over cluster age; this pilot study gives evidence that the X-ray population also changes with time, and demonstrates the need for a broader multiwavelength study of Galactic open clusters.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.08591-b31b1b.svg)](https://arxiv.org/abs/2601.08591) | **Dust Properties of the Interstellar Object 3I/ATLAS Revealed by Optical and Near-Infrared Polarimetry**  |
|| S. Choi, et al. -- incl., <mark>K. Doi</mark> |
|*Appeared on*| *2026-01-14*|
|*Comments*| *11 pages, 4 Figures, 3 Tables*|
|**Abstract**|            We present independent polarimetric observations of the interstellar object 3I/ATLAS, including the first near-infrared polarimetric measurements. Using imaging polarimeters, we measured the degree of linear polarization from the visible RC band (0.64 {\mu}m) to the near-infrared KS band (2.25 {\mu}m), and investigated its dependence on solar phase angle (polarization phase curve; PPC) and wavelength (polarization color curve; PCC). We confirm that the PPC of 3I/ATLAS differs significantly from those of typical Solar System comets, showing an unusually large polarization amplitude. This PPC shows no significant change in the RC band across perihelion passage, despite the perihelion lying within the water snow line. This indicates that the unusual polarimetric behavior of 3I/ATLAS is unlikely to be driven by transient volatile activity, but instead reflects intrinsic optical properties of refractory dust particles. The PCC increases with wavelength over 0.6-1.2 {\mu}m and peaks at 1.5-2.0 {\mu}m, suggesting that the dominant scattering units are dust aggregates composed of submicron-sized monomers, broadly consistent with interstellar dust and solar-system cometary aggregates. Taken together, our results indicate that 3I/ATLAS preserves polarimetric properties characteristic of a primitive cometary planetesimal formed in another planetary system, with a refractory dust composition that differs from that typically observed among Solar System comets, despite sharing a similar size scale of the aggregate building blocks.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.07925/./Figures/Fig_LLAMAS.png', 'tmp_2601.07925/./Figures/Fig_LLAMAS.png', 'tmp_2601.07925/./Figures/Fig_gaia_atlas.png']
copying  tmp_2601.07925/./Figures/Fig_LLAMAS.png to _build/html/
copying  tmp_2601.07925/./Figures/Fig_LLAMAS.png to _build/html/
copying  tmp_2601.07925/./Figures/Fig_gaia_atlas.png to _build/html/
exported in  _build/html/2601.07925.md
    + _build/html/tmp_2601.07925/./Figures/Fig_LLAMAS.png
    + _build/html/tmp_2601.07925/./Figures/Fig_LLAMAS.png
    + _build/html/tmp_2601.07925/./Figures/Fig_gaia_atlas.png
found figures ['tmp_2601.08087/./gudelbenz.png', 'tmp_2601.08087/./ic2602cmd.png', 'tmp_2601.08087/./IC2602Xray.png']
copying  tmp_2601.08087/./gudelbenz.png to _build/html/
copying  tmp_2601.08087/./ic2602cmd.png to _build/html/
copying  tmp_2601.08087/./IC2602Xray.png to _build/html/
exported in  _build/html/2601.08087.md
    + _build/html/tmp_2601.08087/./gudelbenz.png
    + _build/html/tmp_2601.08087/./ic2602cmd.png
    + _build

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# An eclipsing 8.56 minute orbital period mass-transferring binary

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.07925-b31b1b.svg)](https://arxiv.org/abs/2601.07925)<mark>Appeared on: 2026-01-14</mark> -  _14 pages, 10 figures_

</div>
<div id="authors">

E. T. Chickles, et al. -- incl., <mark>K. El-Badry</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of ATLAS J101342.5 $-$ 451656.8 (hereafter referred to as ATLAS J1013 $-$ 4516), an 8.56 minute orbital-period mass-transferring AM Canum Venaticorum (AM CVn) binary with a mean _Gaia_ magnitude of $G=19.51$ , identified via periodic variability in light curves from the Asteroid Terrestrial-impact Last Alert System (ATLAS) of _Gaia_ white-dwarf candidates. Follow-up with the Large Lenslet Array Magellan Spectrograph (LLAMAS) shows a helium-dominated accretion disk, and high-speed ULTRACAM photometry reveals pronounced primary and secondary eclipses. We construct a decade-long timing baseline leveraging lightcurves from the ATLAS and _Gaia_ surveys, as well as the high-speed imagers ULTRACAM on the GTC and proto-Lightspeed on the Magellan Clay telescope. From this timing baseline, we measure an orbital period derivative of $\dot{P}=-1.60\pm0.07\times 10^{-12}$ s s $^{-1}$ . Interpreted in the context of stable mass transfer, the magnitude and sign of $\dot{P}$ indicate that the orbital evolution is governed by the interplay between gravitational-wave--driven angular-momentum losses and mass transfer, directly probing the donor's structural response to mass loss. We constrain the accretor and donor mass based on stable mass-transfer arguments assuming angular momentum loss dominated by gravitational wave emission, allowing us to infer the characteristic gravitational wave strain of the binary for future space-based GW observatories such as the Laser Interferometer Space Antenna ( _LISA_ ). We predict a characteristic strain corresponding to a 4-year _LISA_ signal-to-noise ratio $\gtrsim20$ , establishing ATLAS J1013 $-$ 4516 as a strong prospective _LISA_ source that will probe long-term orbital evolution in the mass-transferring regime.

</div>

<div id="div_fig1">

<img src="tmp_2601.07925/./Figures/Fig_LLAMAS.png" alt="Fig15" width="100%"/>

**Figure 15. -** Coadded, continuum-subtracted Magellan/LLAMAS spectra of ATLAS J1013$-$4516 obtained on 2025 December 14. The displayed wavelength range is restricted to the highest signal-to-noise, non--sky-dominated region of the LLAMAS coverage. Right panels show phase-resolved He ii line profiles, constructed by binning the spectra into four equal orbital phase bins and arbitrarily offset vertically for clarity. Both the phase-binned line profiles and the coadded spectrum have been smoothed with a Gaussian kernel for visualization. (*fig:LLAMAS*)

</div>
<div id="div_fig2">

<img src="tmp_2601.07925/./Figures/Fig_LLAMAS.png" alt="Fig18" width="100%"/>

**Figure 18. -** Coadded, continuum-subtracted Magellan/LLAMAS spectra of ATLAS J1013$-$4516 obtained on 2025 December 14. The displayed wavelength range is restricted to the highest signal-to-noise, non--sky-dominated region of the LLAMAS coverage. Right panels show phase-resolved He ii line profiles, constructed by binning the spectra into four equal orbital phase bins and arbitrarily offset vertically for clarity. Both the phase-binned line profiles and the coadded spectrum have been smoothed with a Gaussian kernel for visualization. (*fig:LLAMAS*)

</div>
<div id="div_fig3">

<img src="tmp_2601.07925/./Figures/Fig_gaia_atlas.png" alt="Fig1" width="100%"/>

**Figure 1. -** Phase-folded light curves of ATLAS J1013$-$4516 from synoptic surveys, folded on the timing ephemeris listed in Table \ref{tab:inferred} with phase zero defined at mid-eclipse. Cyan ($c$, 420–650 nm) and orange ($o$, 560–820 nm) points show phase-binned ATLAS photometry, overlaid with *Gaia* epoch photometry. All data are binned using inverse-variance--weighted averaging and repeated over three orbital cycles for visualization. The ATLAS light curves are linearly rescaled to match the interquartile range and median level of the phase-binned *Gaia* photometry. The resulting $\sim$50\% eclipse-like attenuation and coherent periodic modulation motivated high-speed follow-up. (*fig:atlas*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.07925"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$</div>



<div id="title">

# The Secret Lives of Open Clusters: a Multiwavelength Examination of Three Open Clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.08087-b31b1b.svg)](https://arxiv.org/abs/2601.08087)<mark>Appeared on: 2026-01-14</mark> -  _accepted to PASA_

</div>
<div id="authors">

K. C. Dage, et al. -- incl., <mark>A. Patil</mark>

</div>
<div id="abstract">

**Abstract:** Star clusters are well known for their dynamical interactions, an outcome of their high stellar densities; in this paper we use multiwavelength observations to search for the unique outcomes of these interactions in three nearby Galactic open clusters: IC 2602 (30  Myr), NGC 2632 (750 Myr) and M67 (4 Gyr). We compared X-ray observations from all-sky surveys like eROSITA, plus archival observations from _Chandra_ X-ray Observatory, survey radio observations from ASKAP's Evolutionary Map of the Universe survey plus archival VLA observations, in conjunction with new cluster catalogs with Gaia. From X-ray, we found 77 X-ray sources likely associated with IC 2602, 31 X-ray sources in NGC 2632, and 31 near M67's central regions. We were further able to classify these X-ray sources based on their optical variability and any radio emission. Three IC 2602 X-ray sources had radio counterparts, which are likely all chromospherically active binary stars. We also identified luminous radio and X-ray variability from a spectroscopic triple system in M67, WOCS 3012/S1077, which is either consistent with a quiescent black hole binary, or due to an active binary stellar system. A recent population study of optical variables by Anderson \& Hunt 2025 shows that the population of optical variables in open clusters clearly changes over **cluster age** ; this pilot study gives evidence that the X-ray population also changes with time, and demonstrates the need for a broader multiwavelength study of Galactic open clusters.

</div>

<div id="div_fig1">

<img src="tmp_2601.08087/./gudelbenz.png" alt="Fig7" width="100%"/>

**Figure 7. -** The location of IC 2602 radio/X-ray sources (Table 5) and WOCS 3012/S1077 on the relation of radio and X-ray for active binaries from \cite{1995A&A...302..775G}. Adapted from \cite{Paduano24}. The three sources in IC 2602 fall firmly on the correlation for active binaries. WOCS 3012/S1077 (dashed line in left hand side) falls in the scatter near the correlation.  (*fig:gudelbenz*)

</div>
<div id="div_fig2">

<img src="tmp_2601.08087/./ic2602cmd.png" alt="Fig1" width="100%"/>

**Figure 1. -** CMD for IC 2602. Black x's are X-rays from eROSITA, green triangles are $\delta$ Scuti/$\gamma$ Doradus/SX Pheonicis, purple pentagons are ACV systems, red triangles are RS Canum Venaticorum, teal squares are eclipsing binaries and orange points are solar-like variability.  (*fig:ic2602cmd*)

</div>
<div id="div_fig3">

<img src="tmp_2601.08087/./IC2602Xray.png" alt="Fig2" width="100%"/>

**Figure 2. -** X-ray luminosity (eROSITA band) versus absolute G magnitude for X-ray sources in IC 2602. Sources with variability flags from \cite{2023A&A...674A..13E} are labeled in orange pentagons, with periodic sources marked with pink diamonds. Three sources (yellow triangles) had radio emission associated with them, but were not flagged as variable by \cite{2023A&A...674A..13E}.  (*fig:ic2602-var*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.08087"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

114  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
